#### IMPORT THE LIBRARIES

In [76]:
import numpy as np
import cv2
import os
import model as embedding
import torch
import pickle

In [0]:
#upload model.py, deploy.prototxt.txt, res10, dataset.zip

In [0]:
!unzip dataset.zip

Archive:  dataset.zip
   creating: dataset/APJ Abdul Kalam/
  inflating: dataset/APJ Abdul Kalam/kalam-1.jpeg  
  inflating: dataset/APJ Abdul Kalam/kalam-10.jpg  
  inflating: dataset/APJ Abdul Kalam/kalam-2.jpeg  
  inflating: dataset/APJ Abdul Kalam/kalam-3.jpeg  
  inflating: dataset/APJ Abdul Kalam/kalam-4.jpeg  
  inflating: dataset/APJ Abdul Kalam/kalam-5.jpeg  
  inflating: dataset/APJ Abdul Kalam/kalam-6.jpg  
  inflating: dataset/APJ Abdul Kalam/kalam-7.jpg  
  inflating: dataset/APJ Abdul Kalam/kalam-8.jpg  
  inflating: dataset/APJ Abdul Kalam/kalam-9.jpg  
   creating: dataset/Ashwin/
  inflating: dataset/Ashwin/ashwin-1.jpeg  
  inflating: dataset/Ashwin/ashwin-10.jpg  
  inflating: dataset/Ashwin/ashwin-3.jpg  
  inflating: dataset/Ashwin/ashwin-4.jpg  
  inflating: dataset/Ashwin/ashwin-5.jpg  
  inflating: dataset/Ashwin/ashwin-6.jpg  
  inflating: dataset/Ashwin/ashwin-7.jpg  
  inflating: dataset/Ashwin/ashwin-8.jpg  
  inflating: dataset/Ashwin/ashwin-9.jpg  
   cre

In [77]:
# face detection model paths
protoPath = os.getcwd()+"/deploy.prototxt.txt"
modelPath = os.getcwd()+"/res10_300x300_ssd_iter_140000.caffemodel"

# loading detection model
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

# load embedding model
embedder = embedding.InceptionResnetV1(pretrained='vggface2').eval()

# paths to save pickle files
currentDir = os.getcwd()

In [78]:
# images folder
dataset = os.path.join(currentDir, "dataset")

# paths to save pickle files
!mkdir output
!touch  output/SimpleEmbeddings.pickle

A subdirectory or file output already exists.


Touching output/SimpleEmbeddings.pickle


In [79]:
# getting all images paths

imagePaths = []

for person in os.listdir(dataset):
    for img in os.listdir(dataset+"/"+person):
        imagePaths.append(dataset+"/"+person+"/"+img)

In [80]:
# create lists to append ImgPaths/names/imageIDs/boxs/embeddings
ImgPaths = []
names = []
imageIDs = []
boxs = []
embeddings = []

In [74]:
imagePaths[0].split("/")[-2]

'APJ Abdul Kalam'

In [75]:
names = []
for (i, imagePath) in enumerate(imagePaths):
    name = imagePath.split("/")[-2]
    names.append(name)

In [81]:
# initlize the total number of faces processed
total = 0

# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
  
    #print(i,imagePath)
    
    #extract the person name from the image path
    
    name = imagePath.split("/")[-2]
    imageID = imagePath.split(os.path.sep)[-1].split('.')[-2]
    
    image = cv2.imread(imagePath)
    (h,w) = image.shape[:2]
    
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    
    detector.setInput(blob)
    detections = detector.forward()
    
    if len(detections) > 0:
        
        # we're making the assumption that each image has only ONE
        # face, so find the bounding box with the largest probalility
        
        i = np.argmax(detections[0, 0, :, 2])
        confidence = detections[0, 0, i, 2]
        
        # ensure that the detection with the largest probability also
        # means our minimum probability test (thus helping filter out
        # weak detections)
        
        if confidence > 0.5:
            
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            
            (startX, startY, endX, endY) = box.astype("int")
            
            face = image[startY:endY , startX:endX]
            (fH , fW) = face.shape[:2]
            
            
            # ensure the facce width and height are sufficently large
            if fW < 20 or fH < 20:
                continue
                
            try:
                faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,(160, 160), (0, 0, 0), swapRB=True, crop=False)
            except:
                print("[Error] - Face size in Image not sufficent to get Embeddings : ", imagePath)
                continue
            
            faceTensor = torch.tensor(faceBlob)
            faceEmbed = embedder(faceTensor)
            flattenEmbed = faceEmbed.squeeze(0).detach().numpy()
            
            ImgPaths.append(imagePath)
            imageIDs.append(imageID)
            names.append(name)
            boxs.append(box)
            embeddings.append(flattenEmbed)
            total += 1

In [82]:
names

['APJ Abdul Kalam',
 'APJ Abdul Kalam',
 'APJ Abdul Kalam',
 'APJ Abdul Kalam',
 'APJ Abdul Kalam',
 'APJ Abdul Kalam',
 'APJ Abdul Kalam',
 'APJ Abdul Kalam',
 'APJ Abdul Kalam',
 'APJ Abdul Kalam',
 'Ashwin',
 'Ashwin',
 'Ashwin',
 'Ashwin',
 'Ashwin',
 'Ashwin',
 'Ashwin',
 'Ashwin',
 'Ashwin',
 'B kumar',
 'B kumar',
 'B kumar',
 'B kumar',
 'B kumar',
 'B kumar',
 'B kumar',
 'B kumar',
 'bumrah',
 'bumrah',
 'bumrah',
 'bumrah',
 'bumrah',
 'bumrah',
 'bumrah',
 'bumrah',
 'bumrah',
 'dhawan',
 'dhawan',
 'dhawan',
 'dhawan',
 'dhawan',
 'dhawan',
 'dhawan',
 'dhawan',
 'dhawan',
 'dhawan',
 'dhoni',
 'dhoni',
 'dhoni',
 'dhoni',
 'dhoni',
 'dhoni',
 'dhoni',
 'dhoni',
 'dhoni',
 'dhoni',
 'DYPatil',
 'DYPatil',
 'DYPatil',
 'DYPatil',
 'DYPatil',
 'DYPatil',
 'DYPatil',
 'DYPatil',
 'DYPatil',
 'DYPatil',
 'jadeja',
 'jadeja',
 'jadeja',
 'jadeja',
 'jadeja',
 'jadeja',
 'jadeja',
 'jadeja',
 'jadeja',
 'jadeja',
 'modi',
 'modi',
 'modi',
 'modi',
 'modi',
 'modi',
 'modi',
 'm

In [83]:
# dump the facial embeddings + names to disk
print("[INFO] serializing {} encodings ....".format(total))
data = {"paths":ImgPaths, "names":names, "imageIDs":imageIDs, "boxs":boxs, "embeddings":embeddings}
file_path = "F:/FACENET/output/SimpleEmbeddings.pickle"
with open(file_path , "wb") as f:
    pickle.dump(data,f)

[INFO] serializing 183 encodings ....


In [84]:
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
import pickle
import numpy as np

# paths to embedding pickle file
embeddingPickle = "./output/SimpleEmbeddings.pickle"

# path to recognizer pickle file
!touch output/SimpleRecognizer.pickle
recognizerPickle = "./output/SimpleRecognizer.pickle"

# path to labels pickle file
!touch output/SimpleLabel.pickle
labelPickle = "./output/SimpleLabel.pickle"

# loading embeddings pickle
data = pickle.loads(open(embeddingPickle, "rb").read())

# encode the labels
label = LabelEncoder()
labels = label.fit_transform(data["names"])

# getting embeddings
Embeddings = np.array(data["embeddings"])

# train the model used to accept the 512-d embeddings of the face and 
# then produce the actual face recognition

recognizer = KNeighborsClassifier(n_neighbors= 2, metric='euclidean', weights="distance")
#recognizer = SVC(probability=True)
recognizer.fit(Embeddings, labels)

# write the actual face recognition model to disk
f = open(recognizerPickle, "wb")
f.write(pickle.dumps(recognizer))
f.close()

# write the label encoder to disk
f = open(labelPickle,"wb")
f.write(pickle.dumps(label))
f.close()

Touching output/SimpleRecognizer.pickle
Touching output/SimpleLabel.pickle


In [85]:
# loading face detection model
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

# load embedding model
embedder = embedding.InceptionResnetV1(pretrained="vggface2").eval()

# load the actual face recognition model along with the label encoder
recognizer = pickle.loads(open(recognizerPickle, "rb").read())
label = pickle.loads(open(labelPickle, "rb").read())

In [88]:
imagePath = os.getcwd() + "/test3.jpg"

predictedImg = os.getcwd()

image = cv2.imread(imagePath)
(h,w) = image.shape[:2]

blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

detector.setInput(blob)
detections = detector.forward()

# loop over the detections
for i in range(0, detections.shape[2]):
    
    # extract the confidence (i.e., probalility) associated with the prediction
    confidence = detections[0, 0, i, 2]
    
    # fillter out weak detections
    if confidence > 0.2:
        
        # compute the (x ,y) - coordinates of the bounding box for the face
        
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        
        # extract the face ROI
        face = image[startY:endY , startX:endX]
        (fH ,fW) = face.shape[:2]
        
        # ensure the facce width and height are sufficently large
        if fW < 20 or fH < 20:
            print("[Error] - Face size in Image not sufficent to get Embeddings : ", imagePath)
            continue
        

        try:
            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,(160, 160), (0, 0, 0), swapRB=True, crop=False)
        except:
            print("[Error] - Face size in Image not sufficent to get Embeddings : ", imagePath)
            continue
        
        faceTensor = torch.tensor(faceBlob)
        faceEmbed = embedder(faceTensor)
        flattenEmbed = faceEmbed.squeeze(0).detach().numpy()
        
        array = np.array(flattenEmbed).reshape(1,-1)
        
        # perform classification to recognize the face
        
        preds = recognizer.predict_proba(array)[0]
        
        j = np.argmax(preds)
        
        proba = preds[j]
        name = label.classes_[j]
        
        #draw the bunding box of the face along with the associated probability
        
        text = "{}: {:.2f}%".format(name, proba * 100)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        
        cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
        
        cv2.putText(image, text, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.50, (255, 255, 255), 1)
        
# save image predicte folder
cv2.imwrite("{}/test_prediction.png".format(predictedImg), image)

True